In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import os

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

from datetime import date
import holidays
us_holidays = holidays.UnitedStates()

pd.set_option('display.max_rows', 500)

## Merge test set

In [3]:
# test_df = pd.read_feather('Data/test.feather')
# weather_test_df = pd.read_feather('Data/weather_test.feather')
# building_meta_df = pd.read_feather('Data/building_metadata.feather')

# test = test_df.merge(building_meta_df, left_on = "building_id", right_on = "building_id", how = "left")
# test = test.merge(weather_test_df, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how='left')

In [5]:
test.shape

(41697600, 23)

## Summary for next time I look at the data
1. need 41697600 data
2. how do I align all data: key should be 'row_id'

## Note: I name the dataframe as train here because I don't want to change the code below... I am just lazy

#### Take out cloud_coverage, precip_depth_1_hr, wind_direction, and  sea_level_pressure for now

In [6]:
# train = test.drop(['year_built', 'floor_count', 
#                     'precip_depth_1_hr', 'cloud_coverage', 'sea_level_pressure', 'wind_direction',
#                    'precip_depth_1_hr_lag', 'cloud_coverage_lag', 
#                     'sea_level_pressure_lag', 'wind_direction_lag'], axis=1)
# # train = train.dropna()
# train.isnull().sum()/train.shape[0]

row_id                 0.000000
building_id            0.000000
meter                  0.000000
timestamp              0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
air_temperature        0.005322
dew_temperature        0.006255
wind_speed             0.007245
air_temperature_lag    0.005325
dew_temperature_lag    0.006258
wind_speed_lag         0.007248
dtype: float64

## Prepocess
1. Label primary_use 
2. move y to the first column
3. Add month, week day, day, hour, holidays

In [7]:
# %%time
# le = preprocessing.LabelEncoder()
# train['primary_use'] = le.fit_transform(train['primary_use'])

# move = ['timestamp'] + [x for x in train.columns if x not in ['meter_reading', 'timestamp']]
# train = train[move]

# # mask =(train['site_id']==0) & (train['timestamp'] < '2016-06-01')
# # train = train[~mask]

# ## this may take mins
# train['date_month'] = train['timestamp'].map(lambda x: x.month)
# train['date_day'] = train['timestamp'].map(lambda x: x.day)
# train['iso_week'] = train['timestamp'].map(lambda x: x.isocalendar()[1])
# train['iso_weekend'] = train['timestamp'].map(lambda x: x.isocalendar()[2])
# train['hour'] = train['timestamp'].map(lambda x: x.hour)
# train['holidays'] = train['timestamp'].map(lambda x: 1 if x in us_holidays else 0)
# train['holidays'] = train['timestamp'].map(lambda x: 1 if x in us_holidays else 0)

NameError: name 'us_holidays' is not defined

In [10]:
# train.to_feather('Data/test_joined_processed.feather')

In [2]:
train = pd.read_feather('Data/test_joined_processed.feather')

In [4]:
train.shape

(41697600, 19)

## Get building_list

In [10]:
# building_list = train['building_id'].drop_duplicates()
# len(building_list)

1449

## Run models on test data

In [3]:
%%time
prediction = pd.DataFrame()

path = 'models2'

# sample_df = train[train['site_id'] ==0 ]
# site_0_variables = ['building_id', 'primary_use', 'square_feet', 'air_temperature', 'dew_temperature',
#        'wind_speed', 'air_temperature_lag', 'dew_temperature_lag',
#        'wind_speed_lag', 'date_month', 'date_day', 'iso_week', 'iso_weekend',
#        'hour', 'holidays']

# for meter in sample_df['meter'].value_counts().index.values:
#     print('site0 meter{}'.format(meter))
    
#     X = sample_df[sample_df['meter'] == meter ][site_0_variables]
#     bst = lgb.Booster(model_file = path + '/model_site0_meter{}.txt'.format(meter))
    
#     prediction_on_X_train = bst.predict(X)
#     prediction_on_X_train = pd.DataFrame(prediction_on_X_train, columns=['meter_reading'])
#     prediction_on_X_train[prediction_on_X_train['meter_reading']<0] = 0
#     prediction_on_X_train['row_id'] = sample_df[sample_df['meter'] == meter ].iloc[:,1].values
#     prediction_on_X_train = prediction_on_X_train[['row_id', 'meter_reading']]
    
#     prediction = pd.concat([prediction, prediction_on_X_train])
    ####################################################################



site_variables = [
       'primary_use', 'square_feet', 'air_temperature', 'dew_temperature',
       'wind_speed', 'air_temperature_lag', 'dew_temperature_lag',
       'wind_speed_lag', 'date_month', 'date_day', 'iso_week', 'iso_weekend',
       'hour', 'holidays']

for building in range(0,1449):
    sample_df = train[train['building_id'] == building]
    for meter in sample_df['meter'].value_counts().index.values:
        print('site{} meter{}'.format(building, meter))
        
        X = sample_df[sample_df['meter'] == meter ][site_variables]
        bst = lgb.Booster(model_file = path + '/model_building{}_meter{}.txt'.format(building, meter))

        prediction_on_X_train = bst.predict(X)
        prediction_on_X_train = pd.DataFrame(prediction_on_X_train, columns=['meter_reading'])
        prediction_on_X_train[prediction_on_X_train['meter_reading']<0] = 0
        prediction_on_X_train['row_id'] = sample_df[sample_df['meter'] == meter ].iloc[:,1].values
        prediction_on_X_train = prediction_on_X_train[['row_id', 'meter_reading']]

        prediction = pd.concat([prediction, prediction_on_X_train])


site0 meter0
site1 meter0
site2 meter0
site3 meter0
site4 meter0
site5 meter0
site6 meter0
site7 meter1
site7 meter0
site8 meter0
site9 meter1
site9 meter0
site10 meter0
site11 meter0
site12 meter0
site13 meter1
site13 meter0
site14 meter1
site14 meter0
site15 meter1
site15 meter0
site16 meter0
site17 meter0
site18 meter0
site19 meter0
site20 meter0
site21 meter0
site22 meter0
site23 meter0
site24 meter0
site25 meter0
site26 meter0
site27 meter0
site28 meter1
site28 meter0
site29 meter1
site29 meter0
site30 meter1
site30 meter0
site31 meter1
site31 meter0
site32 meter0
site33 meter0
site34 meter0
site35 meter0
site36 meter0
site37 meter0
site38 meter0
site39 meter0
site40 meter0
site41 meter0
site42 meter0
site43 meter1
site43 meter0
site44 meter0
site45 meter0
site46 meter0
site47 meter0
site48 meter0
site49 meter0
site50 meter1
site50 meter0
site51 meter0
site52 meter0
site53 meter0
site54 meter0
site55 meter1
site55 meter0
site56 meter0
site57 meter0
site58 meter0
site59 meter0
site

site366 meter0
site367 meter0
site368 meter0
site369 meter0
site370 meter0
site371 meter0
site372 meter0
site373 meter0
site374 meter0
site375 meter0
site376 meter0
site377 meter0
site378 meter0
site379 meter0
site380 meter0
site381 meter0
site382 meter0
site383 meter0
site384 meter0
site385 meter0
site386 meter0
site387 meter0
site388 meter0
site389 meter0
site390 meter0
site391 meter0
site392 meter0
site393 meter0
site394 meter0
site395 meter0
site396 meter0
site397 meter0
site398 meter0
site399 meter0
site400 meter0
site401 meter0
site402 meter0
site403 meter0
site404 meter0
site405 meter0
site406 meter0
site407 meter0
site408 meter0
site409 meter0
site410 meter0
site411 meter0
site412 meter0
site413 meter0
site414 meter0
site415 meter0
site416 meter0
site417 meter0
site418 meter0
site419 meter0
site420 meter0
site421 meter0
site422 meter0
site423 meter0
site424 meter0
site425 meter0
site426 meter0
site427 meter0
site428 meter0
site429 meter0
site430 meter0
site431 meter0
site432 me

site850 meter0
site851 meter0
site852 meter0
site853 meter0
site854 meter0
site855 meter0
site856 meter0
site857 meter0
site858 meter0
site859 meter0
site860 meter0
site861 meter0
site862 meter0
site863 meter0
site864 meter0
site865 meter0
site866 meter0
site867 meter0
site868 meter0
site869 meter0
site870 meter0
site871 meter0
site872 meter0
site873 meter0
site874 meter1
site874 meter0
site875 meter2
site875 meter1
site875 meter0
site876 meter2
site876 meter0
site877 meter0
site878 meter2
site878 meter1
site878 meter0
site879 meter2
site879 meter1
site879 meter0
site880 meter2
site880 meter1
site880 meter0
site881 meter0
site882 meter0
site883 meter1
site883 meter0
site884 meter0
site885 meter2
site885 meter1
site885 meter0
site886 meter2
site886 meter1
site886 meter0
site887 meter2
site887 meter1
site887 meter0
site888 meter2
site888 meter1
site888 meter0
site889 meter2
site889 meter1
site889 meter0
site890 meter2
site890 meter1
site890 meter0
site891 meter1
site891 meter0
site892 me

site1121 meter2
site1121 meter1
site1121 meter0
site1122 meter1
site1122 meter0
site1123 meter0
site1124 meter2
site1124 meter0
site1125 meter0
site1126 meter2
site1126 meter0
site1127 meter1
site1127 meter0
site1128 meter0
site1129 meter2
site1129 meter1
site1129 meter0
site1130 meter1
site1130 meter0
site1131 meter0
site1132 meter1
site1133 meter2
site1133 meter1
site1133 meter0
site1134 meter0
site1135 meter0
site1136 meter1
site1136 meter0
site1137 meter1
site1137 meter0
site1138 meter0
site1139 meter2
site1139 meter1
site1139 meter0
site1140 meter2
site1140 meter1
site1140 meter0
site1141 meter0
site1142 meter1
site1142 meter0
site1143 meter2
site1143 meter1
site1143 meter0
site1144 meter1
site1144 meter0
site1145 meter2
site1146 meter1
site1146 meter0
site1147 meter2
site1147 meter0
site1148 meter2
site1148 meter1
site1148 meter0
site1149 meter2
site1149 meter0
site1150 meter2
site1150 meter1
site1150 meter0
site1151 meter2
site1151 meter1
site1151 meter0
site1152 meter2
site1152

site1332 meter0
site1333 meter1
site1333 meter0
site1334 meter1
site1334 meter0
site1335 meter0
site1336 meter2
site1336 meter1
site1336 meter0
site1337 meter2
site1337 meter1
site1337 meter0
site1338 meter2
site1338 meter0
site1339 meter2
site1339 meter1
site1339 meter0
site1340 meter0
site1341 meter2
site1341 meter1
site1341 meter0
site1342 meter2
site1342 meter1
site1342 meter0
site1343 meter2
site1343 meter1
site1343 meter0
site1344 meter2
site1344 meter1
site1344 meter0
site1345 meter2
site1345 meter1
site1345 meter0
site1346 meter2
site1346 meter0
site1347 meter2
site1347 meter1
site1347 meter0
site1348 meter2
site1348 meter1
site1348 meter0
site1349 meter1
site1350 meter2
site1350 meter1
site1350 meter0
site1351 meter2
site1351 meter1
site1351 meter0
site1352 meter1
site1352 meter0
site1353 meter1
site1353 meter0
site1354 meter2
site1354 meter1
site1355 meter2
site1355 meter1
site1355 meter0
site1356 meter0
site1357 meter1
site1357 meter0
site1358 meter2
site1358 meter1
site1358

In [23]:
# previous code

# %%time
# prediction = pd.DataFrame()

# for sample in building_list: 
#     sample_df = train[train['building_id']==sample]
#     X = sample_df.iloc[:,2:]
    
#     bst = lgb.Booster(model_file = 'models/model{}.txt'.format(sample))
    
#     prediction_on_X_train = bst.predict(X)
#     prediction_on_X_train = pd.DataFrame(prediction_on_X_train, columns=['meter_reading'])
#     prediction_on_X_train[prediction_on_X_train['meter_reading']<0] = 0
#     prediction_on_X_train['row_id'] = sample_df.iloc[:,1].values
#     prediction_on_X_train = prediction_on_X_train[['row_id', 'meter_reading']]
    
#     prediction = pd.concat([prediction, prediction_on_X_train])

Wall time: 6min 41s


In [5]:
prediction.shape

(41697600, 2)

In [9]:
prediction = prediction.sort_values('row_id')
prediction.head(10)

,row_id,meter_reading
0,0,174.471004
1,1,73.424603
2,2,0.000000
3,3,285.713623
4,4,1447.600118
5,5,14.954567
6,6,192.714406
7,7,497.582713
0,8,1302.365821
8,9,535.299030


In [4]:
%%time
prediction.to_csv('submission_bybuilding_site0_bybuilding.csv', index=False)

Wall time: 1min 51s


In [28]:
check = pd.read_csv('submission.csv')
check.head(10)

,row_id,meter_reading
0,0,197.406651
1,1,60.788927
2,2,6.560916
3,3,475.716874
4,4,1456.573171
5,5,12.902798
6,6,187.414600
7,7,502.050283
8,8,1074.977963
9,9,465.296370
